# SVM Classification

This data set is made available by http://www.baseball-reference.com. It contains data about professional baseball (MLB) games played in the 2016 season. There are 2,427 games in the data set. Each row represents a single game. Predict the attendance at a home team’s game. This is an important task because most franchises want to predict the number of attendees for a variety of reasons including profits.

## Goal

Use the **baseball.csv** data set and build a model to predict **attendance_binary**. 

# Read and Prepare the Data:

## Import Data

In [23]:
import numpy as np
import pandas as pd

np.random.seed(42)

In [24]:
baseball = pd.read_csv("baseball.csv")

#Drop GameID column since it can't be used as a feature
baseball = baseball.drop(['GameID'], axis=1)
baseball.head()

,attendance_binary,previous_attendance,previous_away_team_errors,previous_away_team_hits,previous_away_team_runs,game_type,previous_game_type,previous_home_team_errors,previous_home_team_hits,previous_home_team_runs,game_day,previous_game_day,temperature,wind_speed,sky,previous_game_duration,previous_homewin
0,0,43683,2,6,2,Night Game,Day Game,0,6,6,Wednesday,Monday,55,24,Overcast,2.933333,1
1,0,45785,0,7,2,Night Game,Day Game,0,10,3,Wednesday,Monday,48,7,Unknown,2.800000,1
2,0,48282,0,8,4,Night Game,Day Game,2,4,3,Wednesday,Monday,65,10,Cloudy,3.383333,0
3,0,21830,0,9,6,Day Game,Night Game,0,15,11,Wednesday,Tuesday,77,0,In Dome,3.233333,1
4,0,49289,2,4,2,Night Game,Day Game,1,1,3,Tuesday,Monday,81,12,Cloudy,2.633333,1


## Check for Missing Values

In [25]:
baseball.isna().sum()

attendance_binary            0
previous_attendance          0
previous_away_team_errors    0
previous_away_team_hits      0
previous_away_team_runs      0
game_type                    0
previous_game_type           0
previous_home_team_errors    0
previous_home_team_hits      0
previous_home_team_runs      0
game_day                     0
previous_game_day            0
temperature                  0
wind_speed                   0
sky                          0
previous_game_duration       0
previous_homewin             0
dtype: int64

## Split Data

In [26]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(baseball, test_size=0.3)

## Separate Target

In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

train_y = train[['attendance_binary']]
test_y = test[['attendance_binary']]

train_inputs = train.drop(['attendance_binary'], axis=1)
test_inputs = test.drop(['attendance_binary'], axis=1)

## Identify Numeric and Categorical Columns

In [28]:
train_inputs.dtypes

previous_attendance            int64
previous_away_team_errors      int64
previous_away_team_hits        int64
previous_away_team_runs        int64
game_type                     object
previous_game_type            object
previous_home_team_errors      int64
previous_home_team_hits        int64
previous_home_team_runs        int64
game_day                      object
previous_game_day             object
temperature                    int64
wind_speed                     int64
sky                           object
previous_game_duration       float64
previous_homewin               int64
dtype: object

In [29]:
# Identify the numeric columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns
numeric_columns

Index(['previous_attendance', 'previous_away_team_errors',
       'previous_away_team_hits', 'previous_away_team_runs',
       'previous_home_team_errors', 'previous_home_team_hits',
       'previous_home_team_runs', 'temperature', 'wind_speed',
       'previous_game_duration', 'previous_homewin'],
      dtype='object')

In [30]:
# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns
categorical_columns

Index(['game_type', 'previous_game_type', 'game_day', 'previous_game_day',
       'sky'],
      dtype='object')

## Pipeline

In [31]:
numeric_transformer = Pipeline(steps=[               
                ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[  
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)])

## Transform Train and Test

In [32]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)
train_x.shape

(1698, 37)

In [33]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)
test_x.shape

(729, 37)

# Determine Baseline Accuracy

In [34]:
# Find majority class
train_y['attendance_binary'].value_counts()

1    873
0    825
Name: attendance_binary, dtype: int64

In [35]:
# Find percentage
train_y['attendance_binary'].value_counts()/len(train_y)

1    0.514134
0    0.485866
Name: attendance_binary, dtype: float64

A naive prediction of 1 yields an accuracy of only 51.4%

# SVM Model 1 - SVC(kernel=linear)

In [41]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
 
lin_svm = SVC(kernel="linear")

lin_svm.fit(train_x, train_y)

C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [42]:
#Predict the train values
train_y_pred = lin_svm.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.8445229681978799

In [43]:
#Predict the test values
test_y_pred = lin_svm.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.8161865569272977

A difference in accuracy of fewer than 3 percentage points indicates that there is not much overfitting.

# SVM Model 2 - SVC(kernel=poly)

In [81]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
 
pol_svm = SVC(kernel="poly", degree=2, coef0=1, C=5, gamma='scale')

pol_svm.fit(train_x, train_y)

C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=5, cache_size=200, class_weight=None, coef0=1,
    decision_function_shape='ovr', degree=2, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [82]:
#Predict the train values
train_y_pred = pol_svm.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.8957597173144877

In [83]:
#Predict the test values
test_y_pred = pol_svm.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.8079561042524005

The degree-2 polynomial model performed slightly worse than the linear model, and had noticeably worse overfitting (~9 percentage points).

# SVM Model 3 - SVC(kernel=rbf)

In [75]:
rbf_svm = SVC(kernel="rbf", C=10, gamma=0.1)

rbf_svm.fit(train_x, train_y)

C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [76]:
#Predict the train values
train_y_pred = rbf_svm.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.9929328621908127

In [77]:
#Predict the test values
test_y_pred = rbf_svm.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.7983539094650206

A much larger difference in accuracy of nearly 20 percentage points indicates significant overfitting.

# SVM Model 4 - SVC(rbf) with Grid Search

In [90]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import random

#define ranges for C and gamma
param_distribs = {
        'C': randint(low=5, high=70),
        'gamma': uniform(0.01, 0.5),    
    }

#define model
rbf_svm = SVC(kernel="rbf")

#search variations of hyperparameters 50 iterations
rbf_search = RandomizedSearchCV(rbf_svm, param_distributions=param_distribs,
                                n_iter=50, cv=10, scoring='accuracy', random_state=42)

#fit models on training data
rbf_search.fit(train_x, train_y)

C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning

C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning

C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning

C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning

C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning

...

C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning

C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning

C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning

C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ajohnson\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=SVC(C=1.0, cache_size=200, class_weight=None,
                                 coef0=0.0, decision_function_shape='ovr',
                                 degree=3, gamma='auto_deprecated',
                                 kernel='rbf', max_iter=-1, probability=False,
                                 random_state=None, shrinking=True, tol=0.001,
                                 verbose=False),
                   iid='warn', n_iter=50, n_jobs=None,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001DE1B8F8F48>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001DE1C962888>},
                   pre_dispatch='2*n_jobs', random_state=42, refit=True,
                   return_train_score=False, scoring='accuracy', verbose=0)

In [91]:
pd.DataFrame(rbf_search.cv_results_).sort_values('mean_test_score',ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
36,0.069216,0.001424,0.005086,2.995339e-04,12,0.0127611,"{'C': 12, 'gamma': 0.0127610585618012}",0.818713,0.842105,0.836257,0.829412,0.876471,0.816568,0.828402,0.840237,0.834320,0.852071,0.837456,0.016467,1
18,0.073703,0.000946,0.004986,3.996624e-07,13,0.0179831,"{'C': 13, 'gamma': 0.0179831261101071}",0.818713,0.842105,0.847953,0.829412,0.870588,0.810651,0.840237,0.834320,0.822485,0.852071,0.836867,0.016778,2
7,0.080885,0.001437,0.004988,6.687801e-06,26,0.0135332,"{'C': 26, 'gamma': 0.013533152609858703}",0.812865,0.830409,0.847953,0.829412,0.870588,0.822485,0.846154,0.840237,0.822485,0.846154,0.836867,0.015921,2
21,0.083881,0.004304,0.005480,4.867632e-04,12,0.0271943,"{'C': 12, 'gamma': 0.0271942605576092}",0.801170,0.830409,0.818713,0.829412,0.858824,0.822485,0.846154,0.834320,0.822485,0.840237,0.830389,0.015136,4
16,0.101624,0.003498,0.005388,4.923228e-04,43,0.0166325,"{'C': 43, 'gamma': 0.016632480579933266}",0.801170,0.807018,0.818713,0.823529,0.858824,0.816568,0.834320,0.840237,0.816568,0.828402,0.824499,0.015978,5
19,0.139229,0.002899,0.011071,2.927390e-04,6,0.352117,"{'C': 6, 'gamma': 0.35211651325607846}",0.748538,0.777778,0.824561,0.829412,0.817647,0.781065,0.781065,0.810651,0.781065,0.828402,0.797998,0.026260,6
38,0.138132,0.003552,0.011471,1.198506e-03,12,0.374504,"{'C': 12, 'gamma': 0.37450358402049366}",0.748538,0.760234,0.836257,0.829412,0.811765,0.781065,0.786982,0.810651,0.775148,0.834320,0.797409,0.029910,7
2,0.140119,0.002752,0.011076,5.437079e-04,28,0.335444,"{'C': 28, 'gamma': 0.33544423647442645}",0.748538,0.771930,0.818713,0.823529,0.817647,0.781065,0.786982,0.804734,0.786982,0.828402,0.796820,0.024693,8
47,0.138231,0.002414,0.011370,4.885491e-04,5,0.370865,"{'C': 5, 'gamma': 0.3708647605824366}",0.748538,0.760234,0.836257,0.829412,0.811765,0.781065,0.781065,0.810651,0.775148,0.834320,0.796820,0.030167,8
39,0.138627,0.001733,0.010872,2.996989e-04,37,0.31298,"{'C': 37, 'gamma': 0.3129799873905057}",0.748538,0.766082,0.812865,0.817647,0.817647,0.775148,0.792899,0.798817,0.798817,0.834320,0.796231,0.025127,10


## Determine the accuracy of the best model

In [96]:
#Pick best model and display
final_model = rbf_search.best_estimator_
rbf_search.best_params_

{'C': 12, 'gamma': 0.0127610585618012}

In [97]:
#Test the model accuracy on the train set
train_predictions = final_model.predict(train_x)

accuracy_score(train_y, train_predictions)

0.8769140164899882

In [98]:
#Test the model accuracy on the test set
test_predictions = final_model.predict(test_x)

accuracy_score(test_y, test_predictions)

0.8161865569272977

The grid search choices of a higher C value and lower gamma gave better results.

The best performing model of the four seems to be the first, simple linear model. The test accuracy of that model is 81.6% and had only 3 percentage points of separation from the train accuracy, indicating low overfitting. In contrast, the polynomial model only achieved 80.8% test accuracy and had noticeable overfitting with a 9 percentage point difference to the train accuracy. The first rbf model failed to improve on the linear model, and had major overfitting. To attempt to find more suitable hyperparameters and allow for a lower gamma (to address overfitting), a randomized grid search found a model that yielded a test accuracy of 81.6%; however this still fails to meaningfully improve on the accuracy original linear model, and still has more overfitting.